In [2]:
import scipy.stats as st
from scipy.stats import norm, t, laplace
from scipy import special
from scipy.integrate import quad
from scipy.special import eval_hermite
import numpy as np
from math import factorial, sqrt

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.layouts import row
from bokeh.io import output_notebook
from bokeh.models import HoverTool,LinearAxis, LogAxis, FixedTicker
from bokeh.palettes import Category10
from bokeh.models import Band,Plot
from bokeh.models import Title, Div, Range1d, BoxAnnotation, Label
from bokeh.layouts import gridplot, column

from tqdm import tqdm


In [16]:
import os
import io
import warnings
import logging
from contextlib import redirect_stdout, redirect_stderr
from bokeh.io import export_svg, save, output_file, export_png
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPDF

from bokeh.io.export import get_screenshot_as_png

# os.environ["PATH"] += os.pathsep + '/opt/homebrew/anaconda3/bin/geckodriver'

# # Убираем Bokeh / Selenium логи
# logging.getLogger("bokeh").setLevel(logging.ERROR)
# logging.getLogger("selenium").setLevel(logging.ERROR)

def save_pic(p, file_name):
    p.background_fill_color=None 
    p.border_fill_color=None
    p.output_backend = "svg"
    
    # f = io.StringIO()
    # with redirect_stdout(f), redirect_stderr(f), warnings.catch_warnings():
    #     warnings.simplefilter("ignore")
    try:
        # HTML
        output_file(filename=f"Pic/html/{file_name}.html", title=file_name)
        save(p)
        # print("HTML")

        # SVG через Selenium
        options = Options()
        options.binary_location = "/opt/homebrew/anaconda3/bin/firefox"
        service = Service("/opt/homebrew/anaconda3/bin/geckodriver")
        driver = webdriver.Firefox(service=service, options=options)
        export_svg(p, filename=f"Pic/svg/{file_name}.svg", webdriver=driver)
        driver.quit()
        # print("SVG")

        # PNG через Selenium
        options = Options()
        options.binary_location = "/opt/homebrew/anaconda3/bin/firefox"
        service = Service("/opt/homebrew/anaconda3/bin/geckodriver")
        driver = webdriver.Firefox(service=service, options=options)
        p.output_backend = "canvas"  # PNG лучше работает с canvas
        export_png(p, filename=f"Pic/png/{file_name}.png", webdriver=driver)
        p.output_backend = "svg"  # Возвращаем обратно для SVG
        driver.quit()
        

        # PDF
        drawing = svg2rlg(f"Pic/svg/{file_name}.svg")
        renderPDF.drawToFile(drawing, f"Pic/pdf/{file_name}.pdf")
        # print("PDF")

    except Exception as e:
        print(f"[save_pic] Ошибка экспорта: {e}")

In [17]:
pallets = ['#cadbce', '#8ab6b6', '#4a919e', '#ce6a6b', '#936fa4', '#5874dc', '#ac9170', '#ffae03', '#e07545', '#c13c87']
code_pallets = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
dict_pallets = dict(zip(code_pallets, pallets))

# Нужные функции

In [18]:
class my_pdf(st.rv_continuous):
    def _pdf(self, x, alpha):
        alpha = np.atleast_1d(alpha).flatten()[0]
        n = int(2 * alpha - 1)
        H_n = special.hermite(n)
        phi = norm.pdf(x)
        f = (1 + 2**(-n/2) * H_n(x / np.sqrt(2)))**2 * phi
        norm_const = 1 + factorial(n)
        return np.asarray(f / norm_const, dtype=np.float64)
    
    def _rvs(self, alpha, size=None, random_state=None, tol=1e-14, max_iter=10):
        # --- 1. Начальный диапазон ---
        x_max = max(10, 3 * np.sqrt(alpha))
        success = False

        for _ in range(max_iter):
            xs = np.linspace(-x_max, x_max, 30000)
            pdf_vals = self._pdf(xs, alpha)
            pdf_vals = np.maximum(pdf_vals, 0)
            integral = np.trapz(pdf_vals, xs)

            # --- 2. Проверка нормировки ---
            if abs(1 - integral) < tol:
                success = True
                break
            else:
                # Расширяем диапазон, если интеграл слишком мал
                x_max *= 1.5

        if not success:
            print(f"⚠️ Warning: PDF may not be fully normalized (integral={integral:.6f})")

        # --- 3. Финальная нормировка ---
        pdf_vals /= np.trapz(pdf_vals, xs)
        p = pdf_vals / pdf_vals.sum()

        # --- 4. Инициализация генератора ---
        rng = self._random_state if random_state is None else random_state

        # --- 5. Выборка ---
        return rng.choice(xs, size=size, p=p)
    

def safe_power_ratio(num, den, alpha):
    ratio = num / den
    # знак отдельно, чтобы избежать nan
    sign = np.sign(ratio)
    abs_val = np.abs(ratio)
    return sign * abs_val ** (1 / (2*alpha - 1))

def safe_power_ratio2(num4,  num2, den4, den2):
    ratio_num = num4 - 3 * num2**2
    ratio_den = den4 - 3 * den2**2
    ratio = ratio_num / ratio_den
    return ratio
    


def moment(dist, order, alpha):
    """Вычисляет момент E[X^order] для данного распределения dist."""
    integrand = lambda x: (x**order) * dist.pdf(x, alpha=alpha)
    val, err = quad(integrand, -np.inf, np.inf, limit=600)
    return val


def hermite_moment_variance(dist, alpha):
    """
    Вычисляет дисперсию Var(X^{2α - 1}) для 'эрмитового' распределения my_pdf.

    Var = E[X^{2(2α - 1)}] - (E[X^{2α - 1}])^2
    """
    order1 = 2 * alpha - 1           # порядок первого момента
    order2 = 2 * order1              # порядок для квадрата

    m1 = moment(dist, order1, alpha)
    m2 = moment(dist, order2, alpha)

    var = m2 - m1**2
    return var, m1, m2


# Hermit dist

In [25]:
def plot_distribution_bokeh2(dist, alpha=[1,2,3,10], sample_size=10000, bins=80, title=None):
    p = figure(
        title=title,
        width=500, height=400,
        x_range=(-10, 10),
        y_axis_label='Плотность',
        toolbar_location=None,
        tools=""
    )
    fill_alpha = [1,1,1,0.4]
    for i in range(len(alpha)):
        # --- Генерация выборки ---
        # samples = dist.rvs(alpha=alpha[i], size=sample_size)
        color = dict_pallets[2*(i+1)]

        # # --- Подготовка данных ---
        # hist, edges = np.histogram(samples, bins=bins, density=True)
        xs = np.linspace(-11, 11, 5000)
        pdf_vals = dist.pdf(xs, alpha=alpha[i])

        # # --- Источники данных для Bokeh ---
        # source_hist = ColumnDataSource(data=dict(
        #     left=edges[:-1],
        #     right=edges[1:],
        #     top=hist
        # ))

        source_pdf = ColumnDataSource(data=dict(
            x=xs,
            y=pdf_vals
        ))

        # # --- Гистограмма ---
        # p.quad(bottom=0, top='top', left='left', right='right', source=source_hist,
        #        fill_alpha=0.3, fill_color=color, line_color='white')

        # --- Теоретическая плотность ---
        p.line('x', 'y', source=source_pdf, alpha=fill_alpha[i], line_width=2, color=color, legend_label = f"k = {2*alpha[i]-1}")

    p.legend.location = "top_right"
    p.legend.click_policy = "hide"
    p.legend.background_fill_alpha = 0     # убрать заливку (прозрачный фон)
    p.legend.border_line_alpha = 0         # убрать рамку
    p.yaxis.axis_label_text_font_size = "14pt"

    # --- Показать ---
    # show(p)
    return p

In [26]:
dist = my_pdf(name='hermite_pdf')
p_dist = plot_distribution_bokeh2(dist, alpha=[1,2,3,4], sample_size=10000, bins=80, title=None)
# show(p_dist)

In [27]:
save_pic(p=p_dist, file_name='hermite_pdf_1_7')

In [28]:
dist = my_pdf(name='hermite_pdf_1_3')
p_dist = plot_distribution_bokeh2(dist, alpha=[1,2], sample_size=10000, bins=80, title=None)

In [29]:
save_pic(p=p_dist, file_name='hermite_pdf_1_3')

In [33]:
def plot_variance_vs_alpha(alpha_max=10, delta=1e-4):
    dist = my_pdf(name="my_pdf")
    alphas = [i + 1 for i in range(alpha_max)]

    stds, ns_required = [], []

    for alpha in alphas:
        var, m1, m2 = hermite_moment_variance(dist, alpha)
        std_val = sqrt(var)
        
        n_req = var / delta**2

        stds.append(var)
        ns_required.append(n_req)

    # --- Определяем диапазоны Y ---
    # y_min_left, y_max_left = min(stds)*0.5, max(ns_required)*1.5
    # y_min_right, y_max_right = min(stds)*0.5, max(ns_required)*1.5

    # --- Основной график ---
    p = figure(
        width=500, height=400,
        # title=fr"Dependence of std($X^{{2α-1}}$) and required $n=\mathrm{{Var}}/\delta$ (δ={delta})",
        x_axis_label="k",
        y_axis_type="log",
        # y_range=Range1d(start=y_min_left, end=y_max_left),
        toolbar_location=None,
        tools="pan,wheel_zoom,box_zoom,reset,save"
    )
    # p.yaxis.axis_label_text_color = dict_pallets[4]
    # p.yaxis.axis_label_text_font_size = "10pt"
    p.xaxis.axis_label_text_font_size = "14pt"
    # p.yaxis.major_label_text_color = dict_pallets[4]
    # p.yaxis.major_tick_line_color = dict_pallets[4]
    # p.yaxis.minor_tick_line_color = dict_pallets[4]
    
    k_list = 2*np.array(alphas) - 1
    # --- Левая ось Y ---
    p.line(k_list, stds, line_width=3, color=dict_pallets[4], legend_label = "var(X^k)")
    p.circle(k_list, stds, size=6, color=dict_pallets[4])

    # # --- Правая ось Y ---
    # p.extra_y_ranges = {"n_range": Range1d(start=y_min_left, end=y_max_right)}
    # right_axis = LogAxis(y_range_name="n_range")
    # right_axis.axis_label_text_color = dict_pallets[3]   # <- цвет подписи
    # right_axis.axis_label_text_font_size = "10pt" # <- размер шрифта
    # right_axis.major_label_text_color = dict_pallets[3]
    # right_axis.major_tick_line_color = dict_pallets[3]      # тики
    # right_axis.minor_tick_line_color = dict_pallets[3]      # второстепенные тики
    # p.add_layout(right_axis, "right")

    # p.line(
    #     k_list, ns_required,
    #     line_width=3, color=dict_pallets[3],
    #     # y_range_name="n_range", 
    #     legend_label = "Var/δ^2"
    # )
    # p.circle(
    #     k_list, ns_required,
    #     size=6, color=dict_pallets[3],
    #     # y_range_name="n_range"
    # )

    # --- Настройки легенды и сетки ---
    p.legend.location = "top_left"
    p.legend.background_fill_alpha = 0
    p.legend.border_line_alpha = 0
    p.grid.grid_line_alpha = 0.4

    # --- HoverTool для обоих наборов данных ---
    hover1 = HoverTool(tooltips=[("α", "$x"), ("std", "$y")], renderers=[p.renderers[0]])
    hover2 = HoverTool(tooltips=[("α", "$x"), ("n", "$y")], renderers=[p.renderers[-2]])
    p.add_tools(hover1, hover2)
    p.xaxis.ticker = FixedTicker(ticks=[k for k in k_list])

    # show(p)
    return p

In [34]:
p_variance_vs_alpha = plot_variance_vs_alpha(alpha_max=6, delta=1e-4)

In [35]:
save_pic(p = p_variance_vs_alpha, file_name='variance_vs_alpha_2')

# Коэф. асимпт. норм. теорема 4

In [36]:
def KANth4(alpha=1, a_true=1.0, n_max=100_000):
    my_dist = my_pdf(name="my_pdf")
    X = my_dist.rvs(alpha=alpha, size=n_max)
    eps = norm.rvs(size=n_max)
    Y = a_true * X + eps

    Y_alpha = Y**(2*alpha - 1)
    X_alpha = X**(2*alpha - 1)

    Y_alpha_mean = np.mean(Y_alpha)
    X_alpha_mean = np.mean(X_alpha)

    X_alpha_new = (Y_alpha_mean / X_alpha_mean) * X_alpha

    samples = (Y_alpha - X_alpha_new)**2

    var_sampels = np.mean(samples)

    return var_sampels / X_alpha_mean**2

    
KANth4(alpha=4, a_true=1.0, n_max=10_000_000) 

7668591116.855134

In [53]:
1036874965.0766999
23359276.252565235
5988830767587.515
42394799.6524349

50882767716.27019

# Теорема 1

In [38]:
# === Основная функция эксперимента === l(n) = n
def run_experiment1(alpha=1, a_true=1.0, n_max=100_000, steps=50, n_trials=10, y_rang = 0.03):
    my_dist = my_pdf(name="my_pdf")
    n_vals = np.linspace(1000, n_max, steps, dtype=int)
    colors = pallets

    # --- Создаем два графика ---
    p1 = figure(
        width=500, height=400,
        # title=f"k={2*alpha-1}",
        x_axis_label="Размер выборки n", y_axis_label=r"Оценка a",
        tools="pan,wheel_zoom,box_zoom,reset,save",
        toolbar_location=None,
        # y_range = (1-y_rang,1+y_rang),
        active_drag="box_zoom"
    )
    p1.add_tools(HoverTool(tooltips=[("n", "@x"), ("a_hat", "@y")]))
    p1.grid.grid_line_alpha = 0.4
    p1.grid.minor_grid_line_alpha = 0.1
    p1.xaxis.axis_label_text_font_size = "14pt"
    p1.yaxis.axis_label_text_font_size = "14pt"

    p2 = figure(
        width=500, height=400,
        # title=fr"Convergence of $a_least_squares$ to a=1 (α={alpha})",
        x_axis_label="Размер выборки n",
        y_axis_label=r"Оценка МНК a",
        tools="pan,wheel_zoom,box_zoom,reset,save",
        toolbar_location=None,
        # y_range = (1-y_rang,1 + y_rang),
        active_drag="box_zoom"
    )
    p2.add_tools(HoverTool(tooltips=[("n", "@x"), ("moment", "@y")]))
    p2.grid.grid_line_alpha = 0.4
    p2.grid.minor_grid_line_alpha = 0.1
    p2.xaxis.axis_label_text_font_size = "14pt"
    p2.yaxis.axis_label_text_font_size = "14pt"

    # --- Эксперимент ---
    for trial in tqdm(range(n_trials)):
        color = colors[trial % len(colors)]

        X = my_dist.rvs(alpha=alpha, size=n_max)
        # X = laplace.rvs(loc=2, size=n_max)
        # eps = t.rvs(df=2*alpha)
        eps = norm.rvs(size=n_max)
        Y = a_true * X + eps

        a_ms, a_hats = [], []

        for n in n_vals:
            Xn = X[:n]
            Yn = Y[:n]

            num = np.mean(Yn**(2*alpha - 1))
            den = np.mean(Xn**(2*alpha - 1))

            a_hat = safe_power_ratio(num, den, alpha)
            a_hats.append(a_hat)
            
            a_m = np.sum(Xn * Yn)/np.sum(Xn * Xn)
            a_ms.append(a_m)

        # --- Рисуем траектории ---
        p1.line(n_vals, a_hats, line_width=1.5, alpha=0.8, color=color)
        p2.line(n_vals, a_ms, line_width=1.5, alpha=0.8, color=color)

    # --- Истинное значение ---
    p1.line([n_vals[0], n_vals[-1]], [a_true, a_true],
            color=pallets[-2], line_dash="dashed", line_width=2, alpha = 0.7)
    
    p2.line([n_vals[0], n_vals[-1]], [a_true, a_true],
            color=pallets[-2], line_dash="dashed", line_width=2, alpha = 0.7)

    # --- Общая компоновка ---
    layout = row(p1, p2)
    # show(layout)
    return layout
    

def run_experiment1_with_ci(alpha=1, a_true=1.0, n_max=100_000, steps=50,
                           n_trials=10, n_ci=30, ci_level=0.9, y_rang = 0.03):
    """
    Строит графики как run_experiment1, но поверх добавляет доверительные интервалы
    по эмпирическим квантилям уровня ci_level.
    """

    # --- Генерация набора траекторий для квантилей ---
    my_dist = my_pdf(name="my_pdf")
    n_vals = np.linspace(1000, n_max, steps, dtype=int)
    all_trajectories = np.zeros((n_ci, len(n_vals)))
    all_a_ls = np.zeros((n_ci, len(n_vals)))

    for ci_trial in tqdm(range(n_ci), desc="CI simulations"):
        X = my_dist.rvs(alpha=alpha, size=n_max)
        eps = norm.rvs(size=n_max)
        Y = a_true * X + eps

        a_hats = []
        a_ls = []
        
        for n in n_vals:
            Xn = X[:n]
            Yn = Y[:n]
            num = np.mean(Yn**(2*alpha - 1))
            den = np.mean(Xn**(2*alpha - 1))
            a_hat = safe_power_ratio(num, den, alpha)
            a_hats.append(a_hat)
            
            a_m = np.sum(Xn * Yn) / np.sum(Xn * Xn)
            a_ls.append(a_m)
            
        all_trajectories[ci_trial, :] = a_hats
        all_a_ls[ci_trial, :] = a_ls

    # --- Квантильные интервалы ---
    lower_q = (1 - ci_level) / 2
    upper_q = 1 - lower_q

    q_lower = np.quantile(all_trajectories, lower_q, axis=0)
    q_upper = np.quantile(all_trajectories, upper_q, axis=0)
    
    q_lower_ls = np.quantile(all_a_ls, lower_q, axis=0)
    q_upper_ls = np.quantile(all_a_ls, upper_q, axis=0)

    # --- Запуск основной функции (она создаёт графики p1, p2) ---
    layout = run_experiment1(alpha=alpha, a_true=a_true,
                             n_max=n_max, steps=steps, n_trials=n_trials, y_rang = y_rang)

    # --- Извлекаем первый график из layout (p1) ---
    p1, p2 = layout.children

    # --- Добавляем на p1 полосу доверительного интервала ---
    source_band = ColumnDataSource(data=dict(
        x=n_vals,
        lower=q_lower,
        upper=q_upper
    ))

    band = Band(base='x', lower='lower', upper='upper', source=source_band,
                level='underlay', fill_alpha=0.2, fill_color=pallets[-1],
                line_width=1, line_color=pallets[-1])

    p1.add_layout(band)
    
    # --- Доверительный интервал для least squares ---
    source_band_ls = ColumnDataSource(data=dict(
        x=n_vals, lower=q_lower_ls, upper=q_upper_ls
    ))
    band_ls = Band(base='x', lower='lower', upper='upper',
                   source=source_band_ls, level='underlay',
                   fill_alpha=0.2, fill_color=pallets[-1],
                   line_width=1, line_color=pallets[-1])
    p2.add_layout(band_ls)

    # show(layout)
    return layout

In [39]:
layout = run_experiment1_with_ci(alpha=1, a_true=1.0, n_max=100_000, steps=100,
                           n_trials=5, n_ci=1000, ci_level=0.95)
p1,p2 = layout.children
save_pic(p = p1, file_name = "a_mean_alpha1")
save_pic(p = p2, file_name = "a_ls_alpha1")

100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 37.50it/s]


In [40]:
layout = run_experiment1_with_ci(alpha=2, a_true=1.0, n_max=100_000, steps=100,
                           n_trials=5, n_ci=1000, ci_level=0.95, y_rang = .1)

p1,p2 = layout.children
save_pic(p = p1, file_name = "a_mean_alpha2")
save_pic(p = p2, file_name = "a_ls_alpha2")

100%|█████████████████████████████████████████████| 5/5 [00:00<00:00,  7.63it/s]


In [41]:
layout = run_experiment1_with_ci(alpha=4, a_true=1.0, n_max=100_000, steps=100,
                           n_trials=5, n_ci=1000, ci_level=0.95, y_rang = 3)

p1,p2 = layout.children
save_pic(p = p1, file_name = "a_mean_alpha4")
save_pic(p = p2, file_name = "a_ls_alpha4")

100%|█████████████████████████████████████████████| 5/5 [00:00<00:00,  7.53it/s]


## Копулы

In [42]:
from scipy.stats import ecdf
from bokeh.models import ColumnDataSource

def copula(alpha=1, n_max=10_000, a_true=1, order = 1):
    # --- 1. Генерация данных ---
    my_dist = my_pdf(name="my_pdf")  # твоя функция для выборки
    X = my_dist.rvs(alpha=alpha, size=n_max)
    eps = norm.rvs(size=n_max)
    Y = a_true * X + eps
    
    k = order
    X_k = X**k
    Y_k = Y**k
    
    # --- 2. Эмпирические функции распределения ---
    F_x = ecdf(X_k)
    F_y = ecdf(Y_k)

    # --- 3. Копульные координаты ---
    U = F_x.cdf.evaluate(X)
    V = F_y.cdf.evaluate(Y)

    # --- 3a. Случайная подвыборка для визуализации ---
    plot_size = 10_000  # количество точек на графике
    indices = np.random.choice(len(U), size=plot_size, replace=False)
    U_plot = U[indices]
    V_plot = V[indices]

    source = ColumnDataSource(data=dict(U=U_plot, V=V_plot))

    # --- 7. Рисуем Bokeh plot ---
    p = figure(
        width=400, height=400,
        x_axis_label = f"F_X^{k}(x)",
        y_axis_label = f"F_Y^{k}(x)",
        tools="pan,wheel_zoom,box_zoom,reset,save",
        toolbar_location = None,
        match_aspect=True
    )

    p.scatter('U', 'V', source=source, size=3, alpha=0.6, color=pallets[2])

    # --- 9. Настройки ---
    p.grid.grid_line_alpha = 0.4
    p.xaxis.axis_label_text_font_size = "14pt"
    p.yaxis.axis_label_text_font_size = "14pt"

    show(p)
    return p

In [43]:
alpha=4
order = 1
p = copula(alpha=alpha, n_max=10_000_000, a_true=1, order = order)

save_pic(p = p, file_name='copula_alpha4_order1')

In [171]:
alpha=4
order = 3
p = copula(alpha=alpha, n_max=10_000_000, a_true=1, order = order)

save_pic(p = p, file_name='copula_alpha4_order3')

In [172]:
alpha=4
order = 5
p = copula(alpha=alpha, n_max=10_000_000, a_true=1, order = order)

save_pic(p = p, file_name='copula_alpha4_order5')

In [44]:
alpha=4
order = 7
p = copula(alpha=alpha, n_max=10_000_000, a_true=1, order = order)

save_pic(p = p, file_name='copula_alpha4_order7')

# Теоремы 1 и 4

In [46]:
# === базовая функция (без доверительных интервалов) ===
def run_experiment2_base(alpha=1, a_true=1.0, n_max=40_000, steps=50, n_trials=5, l_ratio=0.5):
    """
    Возвращает:
      layout (row из двух графиков),
      a_hats_all (список списков оценок для каждого trial),
      n_vals (массив размеров выборки)
    """
    my_dist = my_pdf(name="my_pdf")
    n_vals = np.linspace(1000, n_max, steps, dtype=int)
    colors = pallets

    # --- Графики ---
    p1 = figure(
        width=500, height=400,
        # title=fr"Convergence of $\hat{{a}}_n$ to a={a_true} (α={alpha})",
        x_axis_label="Размер выборки n", y_axis_label=r"Оценка a",
        tools="pan,wheel_zoom,box_zoom,reset,save",
        toolbar_location = None,
        active_drag="box_zoom"
    )
    p1.add_tools(HoverTool(tooltips=[("n", "@x"), ("a_hat", "@y")]))
    p1.grid.grid_line_alpha = 0.4
    p1.xaxis.axis_label_text_font_size = "14pt"
    p1.yaxis.axis_label_text_font_size = "14pt"

    p2 = figure(
        width=500, height=400,
        # title="(empty)",
        x_axis_label="Оценка a", y_axis_label="Гистограмма",
        tools="pan,wheel_zoom,box_zoom,reset,save",
        toolbar_location = None,
        active_drag="box_zoom"
    )
    p2.grid.grid_line_alpha = 0.4
    p2.xaxis.axis_label_text_font_size = "14pt"
    p2.yaxis.axis_label_text_font_size = "14pt"

    l_n = int(l_ratio * n_max)
    l_n = max(1, min(l_n, n_max))

    # --- Эксперимент ---
    for trial in tqdm(range(n_trials)):
        color = colors[trial % len(colors)]

        X_dep = my_dist.rvs(alpha=alpha, size=l_n)
        eps_dep = norm.rvs(size=l_n)
        Y_dep = a_true * X_dep + eps_dep
        
        X_ind = my_dist.rvs(alpha=alpha, size=n_max - l_n)
        Y_ind = a_true * my_dist.rvs(alpha=alpha, size=n_max - l_n) + norm.rvs(size=n_max - l_n)
        
        X_full = np.concatenate([X_dep, X_ind])
        Y_full = np.concatenate([Y_dep, Y_ind])
        
        X_full = np.random.permutation(X_full)
        Y_full = np.random.permutation(Y_full)

        a_hats = []

        for n in n_vals:
            Xn = X_full[:n]
            Yn = Y_full[:n]

            num = np.mean(Yn ** (2 * alpha - 1))
            den = np.mean(Xn ** (2 * alpha - 1))

            a_hat = safe_power_ratio(num, den, alpha)
            a_hats.append(a_hat)

        p1.line(n_vals, a_hats, line_width=2, alpha=0.8, color=color)

    p1.line([n_vals[0], n_vals[-1]], [a_true, a_true],
            color=pallets[-2], line_dash="dashed", line_width=2)

    layout = row(p1, p2)
    return layout

# === Функция с построением доверительного интервала и гистограммы ===
def run_experiment2_with_ci(alpha=13.0, a_true=1.0, n_max=1_000_000, steps=50,
                            n_trials=10, n_ci=30, l_ratio=0.5, ci_level=0.9):
    """
    1. Строит доверительные интервалы (по квантилям эмпирических траекторий).
    2. Рисует гистограмму по последним значениям a_hat из all_trajectories.
    """

    my_dist = my_pdf(name="my_pdf")
    n_vals = np.linspace(1000, n_max, steps, dtype=int)

    all_trajectories = np.zeros((n_ci, len(n_vals)))
    l_n = int(l_ratio * n_max)
    l_n = max(1, min(l_n, n_max))

    # --- Генерация траекторий для доверительного интервала ---
    for ci_trial in tqdm(range(n_ci), desc="CI simulations"):
        X_dep = my_dist.rvs(alpha=alpha, size=l_n)
        eps_dep = norm.rvs(size=l_n)
        Y_dep = a_true * X_dep + eps_dep
        
        X_ind = my_dist.rvs(alpha=alpha, size=n_max - l_n)
        Y_ind = a_true * my_dist.rvs(alpha=alpha, size=n_max - l_n) + norm.rvs(size=n_max - l_n)
        
        X_full = np.concatenate([X_dep, X_ind])
        Y_full = np.concatenate([Y_dep, Y_ind])
        
        X_full = np.random.permutation(X_full)
        Y_full = np.random.permutation(Y_full)

        a_hats = []

        for n in n_vals:
            Xn = X_full[:n]
            Yn = Y_full[:n]

            num = np.mean(Yn ** (2 * alpha - 1))
            den = np.mean(Xn ** (2 * alpha - 1))

            a_hat = safe_power_ratio(num, den, alpha)
            a_hats.append(a_hat)

        all_trajectories[ci_trial, :] = a_hats

    # --- Квантильные интервалы ---
    lower_q = (1 - ci_level) / 2
    upper_q = 1 - lower_q
    q_lower = np.quantile(all_trajectories, lower_q, axis=0)
    q_upper = np.quantile(all_trajectories, upper_q, axis=0)

    # --- Основная фигура ---
    layout = run_experiment2_base(alpha=alpha, a_true=a_true,
                             n_max=n_max, steps=steps, n_trials=n_trials, l_ratio=l_ratio)
    p1, p2 = layout.children

    # --- Добавляем доверительный интервал ---
    source_band = ColumnDataSource(data=dict(
        x=n_vals, lower=q_lower, upper=q_upper
    ))

    band = Band(base='x', lower='lower', upper='upper', source=source_band,
                level='underlay', fill_alpha=0.25, fill_color=pallets[-1],
                line_width=1, line_color=pallets[-1])
    p1.add_layout(band)

    # --- Гистограмма последних оценок ---
    last_vals = all_trajectories[:, -1]

    # --- Гистограмма ---
    hist, edges = np.histogram(last_vals, bins=30, density=True)

    src_hist = ColumnDataSource(dict(
        left=edges[:-1],
        right=edges[1:],
        top=hist
    ))

    p2.quad(bottom=0, top='top', left='left', right='right',
            source=src_hist, fill_alpha=0.5, fill_color=pallets[-1],
            line_color='white')

    # --- Оценка среднего и стандартного отклонения ---
    mean_val = np.mean(last_vals)
    std_val = np.std(last_vals)

    # --- Плавная аппроксимация нормальной плотностью ---
    x_smooth = np.linspace(mean_val - 4*std_val, mean_val + 4*std_val, 400)
    y_smooth = norm.pdf(x_smooth, loc=mean_val, scale=std_val)

    p2.line(x_smooth, y_smooth, color=pallets[-1], line_width=3,
            line_dash="solid")

    # --- Вертикальные линии ---
    p2.line([a_true, a_true], [0, max(hist)], color=pallets[-2],
            line_dash="dashed", line_width=2, legend_label="True a")
    p2.line([mean_val, mean_val], [0, max(hist)], color=pallets[-3],
            line_dash="dotdash", line_width=2, legend_label="Mean \\hat{a}")
    p2.line([mean_val - 3*std_val, mean_val - 3*std_val], [0, max(hist)], color=pallets[-3],
            line_dash="dotted", line_width=2, legend_label="3\\hat{std}")
    p2.line([mean_val + 3*std_val, mean_val + 3*std_val], [0, max(hist)], color=pallets[-3],
            line_dash="dotted", line_width=2, legend_label="3\\hat{std}")

    # --- Красивое оформление ---
    # p2.title.text = "Distribution of final estimates $\\hat{a}$"
    # p2.xaxis.axis_label = "x"
    # p2.yaxis.axis_label = "Density"
    p2.legend.location = "top_right"
    # p2.legend.background_fill_alpha = 0.4
    # p2.legend.label_text_font_size = "10pt"
    p2.toolbar_location = None
    
    p2.legend.background_fill_alpha = 0     # убрать заливку (прозрачный фон)
    p2.legend.border_line_alpha = 0 

    # show(layout)
    return layout


def experiment2_compare_l_ratios(alpha=1, a_true=1.0, n_max=40_000, steps=50,
                     n_trials=5, n_ci=1000, ci_level=0.95,
                                gamma = 0.1, x_min = 0.9, x_max = 1.1, y_max = 70):
    """
    Вызывает run_experiment2_with_ci для двух значений l_ratio (0.9 и 0.01),
    и объединяет результаты горизонтально с синхронизированными осями.
    """

    # --- Первая визуализация (почти зависимые данные) ---
    layout1 = run_experiment2_with_ci(
        alpha=alpha, a_true=a_true,
        n_max=n_max, steps=steps,
        n_trials=n_trials, n_ci=n_ci,
        l_ratio=0.9, ci_level=ci_level
    )

    # --- Вторая визуализация (почти независимые данные) ---
    layout2 = run_experiment2_with_ci(
        alpha=alpha, a_true=a_true,
        n_max=n_max, steps=steps,
        n_trials=n_trials, n_ci=n_ci,
        l_ratio=0.01, ci_level=ci_level
    )

    # --- Извлекаем подграфики ---
    p1a, p1b = layout1.children
    p2a, p2b = layout2.children

    # # --- Синхронизация осей ---
    # p2a.x_range = p1a.x_range
    # p2a.y_range = p1a.y_range
    # p2b.x_range = p1b.x_range
    # p2b.y_range = p1b.y_range
    
    p1a.y_range = Range1d(start = 1 - gamma, end = 1 + gamma)
    p1b.x_range = Range1d(start = x_min, end = x_max)
    p1b.y_range = Range1d(start = 0, end = y_max)
    
    p2a.y_range = Range1d(start = 1 - gamma, end = 1 + gamma)
    p2b.x_range = Range1d(start = x_min, end = x_max)
    p2b.y_range = Range1d(start = 0, end = y_max)
    
    

    # # --- Названия панелей ---
    # p1a.title.text = "High dependence (l_ratio = 0.9)"
    # p2a.title.text = "Low dependence (l_ratio = 0.01)"
    # p1b.title.text = "Final estimates distribution (l_ratio = 0.9)"
    # p2b.title.text = "Final estimates distribution (l_ratio = 0.01)"

    # --- Компоновка: две строки, две колонки ---
    grid = gridplot([[p1a, p2a],
                     [p1b, p2b]],
                    toolbar_location=None,
                    merge_tools=False)

    show(grid)
    return p1a, p1b, p2a, p2b

In [49]:
p1a, p1b, p2a, p2b = experiment2_compare_l_ratios(alpha=1, a_true=1.0, n_max=100_000, steps=50,
                     n_trials=5, n_ci=1500, ci_level=0.95,
                     gamma = 0.06, x_min = 0.95, x_max = 1.05, y_max = 125)
save_pic(p = p1a, file_name = "a_mean_alpha1_z_9")
save_pic(p = p1b, file_name = "a_mean_hist_alpha1_z_9")
save_pic(p = p2a, file_name = "a_mean_alpha1_z_01")
save_pic(p = p2b, file_name = "a_mean_hist_alpha1_z_01")

100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 29.96it/s]


In [48]:
p1a, p1b, p2a, p2b = experiment2_compare_l_ratios(alpha=2, a_true=1.0, n_max=100_000, steps=50,
                     n_trials=5, n_ci=1500, ci_level=0.95,
                     gamma = 0.25, x_min = 0.85, x_max = 1.15, y_max = 25)
save_pic(p = p1a, file_name = "a_mean_alpha2_z_9")
save_pic(p = p1b, file_name = "a_mean_hist_alpha2_z_9")
save_pic(p = p2a, file_name = "a_mean_alpha2_z_01")
save_pic(p = p2b, file_name = "a_mean_hist_alpha2_z_01")

100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 11.58it/s]


# Теорема 2

In [50]:
# === Основная функция эксперимента === l(n) < n
def run_experiment3(alpha=1, a_true=1.0, n_max=40_000, steps=50, n_trials=5, l_ratio=0.5):
    """
    l_ratio ∈ (0, 1] — доля зависимых пар (X,Y)
    """
    my_dist = my_pdf(name="my_pdf")
    n_vals = np.linspace(1000, n_max, steps, dtype=int)
    colors = pallets

    # --- Создаем два графика ---
    p1 = figure(
        width=500, height=400,
        # title=fr"Convergence of $\hat{{sigma^2}}_n$ to sigma^2={1} (α={alpha})",
        x_axis_label="Размер выборки n", y_axis_label=r"Оценка sigma^2",
        toolbar_location = None,
        tools="pan,wheel_zoom,box_zoom,reset,save",
        active_drag="box_zoom"
    )
    p1.add_tools(HoverTool(tooltips=[("n", "@x"), ("a_hat", "@y")]))
    p1.grid.grid_line_alpha = 0.4
    p1.xaxis.axis_label_text_font_size = "14pt"
    p1.yaxis.axis_label_text_font_size = "14pt"
    
    p2 = figure(
        width=500, height=400,
        # title="(empty)",
        x_axis_label="Оценка sigma^2", y_axis_label="Гистограмма",
        toolbar_location = None,
        tools="pan,wheel_zoom,box_zoom,reset,save",
        active_drag="box_zoom"
    )
    p2.grid.grid_line_alpha = 0.4
    p2.xaxis.axis_label_text_font_size = "14pt"
    p2.yaxis.axis_label_text_font_size = "14pt"
    
    l_n = int(l_ratio * n_max)  # число зависимых пар
    l_n = max(1, min(l_n, n_max))  # защита от выхода за пределы

    # --- Эксперимент ---
    for trial in tqdm(range(n_trials)):
        color = colors[trial % len(colors)]
        
        # Чтобы не генерировать заново на каждом шаге n, можно использовать фиксированные массивы
        X_dep = my_dist.rvs(alpha=alpha, size=l_n)
        eps_dep = norm.rvs(size=l_n)
        Y_dep = a_true * X_dep + eps_dep
        
        # независимые X и Y
        X_ind = my_dist.rvs(alpha=alpha, size=n_max - l_n)
        Y_ind = a_true * my_dist.rvs(alpha=alpha, size=n_max - l_n) + norm.rvs(size=n_max - l_n)
        
        # объединяем
        X_full = np.concatenate([X_dep, X_ind])
        Y_full = np.concatenate([Y_dep, Y_ind])
        
        X_full = np.random.permutation(X_full)
        Y_full = np.random.permutation(Y_full)

        sigma2_hats = []

        for n in n_vals:
            
            Xn = X_full[:n]
            Yn = Y_full[:n]

            # --- Вычисление оценок ---
            num = np.mean(Yn ** (2 * alpha - 1))
            den = np.mean(Xn ** (2 * alpha - 1))

            a_hat2 = safe_power_ratio(num, den, alpha)**2
            # a_hats.append(a_hat)
            
            num2 = np.mean(Yn ** 2)
            den2 = np.mean(Xn ** 2)
            
            sigma2_hat = num2 - den2 * a_hat2
            
            sigma2_hats.append(sigma2_hat)


        # --- Рисуем траектории ---
        p1.line(n_vals, sigma2_hats, line_width=2, alpha=0.8, color=color)

    # --- Истинное значение ---
    p1.line([n_vals[0], n_vals[-1]], [1, 1],
            color=pallets[-2], line_dash="dashed", line_width=2)

    # show(p1)
    layout = row(p1, p2)
    return layout



# === Функция с построением доверительного интервала и гистограммы ===
def run_experiment3_with_ci(alpha=1, a_true=1.0, n_max=40_000, steps=50,
                            n_trials=5, n_ci=30, l_ratio=0.5, ci_level=0.9):

    my_dist = my_pdf(name="my_pdf")
    n_vals = np.linspace(1000, n_max, steps, dtype=int)

    all_trajectories = np.zeros((n_ci, len(n_vals)))
    
    l_n = int(l_ratio * n_max)  # число зависимых пар
    l_n = max(1, min(l_n, n_max))  # защита от выхода за пределы

    # --- Генерация траекторий для доверительного интервала ---
    for ci_trial in tqdm(range(n_ci), desc="CI simulations"):
        # Чтобы не генерировать заново на каждом шаге n, можно использовать фиксированные массивы
        X_dep = my_dist.rvs(alpha=alpha, size=l_n)
        eps_dep = norm.rvs(size=l_n)
        Y_dep = a_true * X_dep + eps_dep
        
        # независимые X и Y
        X_ind = my_dist.rvs(alpha=alpha, size=n_max - l_n)
        Y_ind = a_true * my_dist.rvs(alpha=alpha, size=n_max - l_n) + norm.rvs(size=n_max - l_n)
        
        # объединяем
        X_full = np.concatenate([X_dep, X_ind])
        Y_full = np.concatenate([Y_dep, Y_ind])
        
        X_full = np.random.permutation(X_full)
        Y_full = np.random.permutation(Y_full)

        sigma2_hats = []

        for n in n_vals:
            
            Xn = X_full[:n]
            Yn = Y_full[:n]

            # --- Вычисление оценок ---
            num = np.mean(Yn ** (2 * alpha - 1))
            den = np.mean(Xn ** (2 * alpha - 1))

            a_hat2 = safe_power_ratio(num, den, alpha)**2
            # a_hats.append(a_hat)
            
            num2 = np.mean(Yn ** 2)
            den2 = np.mean(Xn ** 2)
            
            sigma2_hat = num2 - den2 * a_hat2
            
            sigma2_hats.append(sigma2_hat)

        all_trajectories[ci_trial, :] = sigma2_hats

    # --- Квантильные интервалы ---
    lower_q = (1 - ci_level) / 2
    upper_q = 1 - lower_q
    q_lower = np.quantile(all_trajectories, lower_q, axis=0)
    q_upper = np.quantile(all_trajectories, upper_q, axis=0)

    # --- Основная фигура ---
    layout = run_experiment3(alpha=alpha, a_true=a_true,
                             n_max=n_max, steps=steps, n_trials=n_trials, l_ratio=l_ratio)
    p1,p2 = layout.children

    # --- Добавляем доверительный интервал ---
    source_band = ColumnDataSource(data=dict(
        x=n_vals, lower=q_lower, upper=q_upper
    ))

    band = Band(base='x', lower='lower', upper='upper', source=source_band,
                level='underlay', fill_alpha=0.25, fill_color=pallets[-1],
                line_width=1, line_color=pallets[-1])
    p1.add_layout(band)
    
    # --- Гистограмма последних оценок ---
    last_vals = all_trajectories[:, -1]
    
    # --- Гистограмма ---
    hist, edges = np.histogram(last_vals, bins=30, density=True)

    src_hist = ColumnDataSource(dict(
        left=edges[:-1],
        right=edges[1:],
        top=hist
    ))

    p2.quad(bottom=0, top='top', left='left', right='right',
            source=src_hist, fill_alpha=0.5, fill_color=pallets[-1],
            line_color='white')

    # --- Оценка среднего и стандартного отклонения ---
    mean_val = np.mean(last_vals)
    std_val = np.std(last_vals)

    # --- Плавная аппроксимация нормальной плотностью ---
    x_smooth = np.linspace(mean_val - 4*std_val, mean_val + 4*std_val, 400)
    y_smooth = norm.pdf(x_smooth, loc=mean_val, scale=std_val)

    p2.line(x_smooth, y_smooth, color=pallets[-1], line_width=3,
            line_dash="solid")

    # --- Вертикальные линии ---
    p2.line([1, 1], [0, max(hist)], color=pallets[-2],
            line_dash="dashed", line_width=2, legend_label="True sigma^2")
    p2.line([mean_val, mean_val], [0, max(hist)], color=pallets[-3],
            line_dash="dotdash", line_width=2, legend_label="Mean hat{sigma^2}")
    p2.line([mean_val - 3*std_val, mean_val - 3*std_val], [0, max(hist)], color=pallets[-3],
            line_dash="dotted", line_width=2, legend_label="3\hat{std}")
    p2.line([mean_val + 3*std_val, mean_val + 3*std_val], [0, max(hist)], color=pallets[-3],
            line_dash="dotted", line_width=2, legend_label="3\hat{std}")

    # --- Красивое оформление ---
    # p2.title.text = "Distribution of final estimates $\\hat{sigma^2}$"
    # p2.xaxis.axis_label = "sigma^2 (final estimates)"
    # p2.yaxis.axis_label = "Density"
    p2.legend.location = "top_right"
    # p2.legend.background_fill_alpha = 0.4
    p2.legend.label_text_font_size = "10pt"
    p2.toolbar_location = None
    
    p2.legend.background_fill_alpha = 0     # убрать заливку (прозрачный фон)
    p2.legend.border_line_alpha = 0 

    # show(layout)
    return layout


def experiment3_compare_l_ratios_sigma(alpha=1, a_true=1.0, n_max=40_000, steps=50,
                                       n_trials=5, n_ci=30, ci_level=0.9,
                                gamma = 0.1, x_min = 0.9, x_max = 1.1, y_max = 70):
    """
    Строит два графика run_experiment3_with_ci с разными l_ratio
    и объединяет их в одну строку с синхронизированной осью Y,
    общими подписями и заголовком (стиль Figure 3 из статьи).
    """

    # --- Левая панель: высокая зависимость ---
    layout1 = run_experiment3_with_ci(
        alpha=alpha, a_true=a_true,
        n_max=n_max, steps=steps,
        n_trials=n_trials, n_ci=n_ci,
        l_ratio=0.9, ci_level=ci_level
    )

    # --- Правая панель: низкая зависимость ---
    layout2 = run_experiment3_with_ci(
        alpha=alpha, a_true=a_true,
        n_max=n_max, steps=steps,
        n_trials=n_trials, n_ci=n_ci,
        l_ratio=0.01, ci_level=ci_level
    )

   # --- Извлекаем подграфики ---
    p1a, p1b = layout1.children
    p2a, p2b = layout2.children

    # # --- Синхронизация осей ---
    # p2a.x_range = p1a.x_range
    # p2a.y_range = p1a.y_range
    # p2b.x_range = p1b.x_range
    # p2b.y_range = p1b.y_range
    
    p1a.y_range = Range1d(start = 1 - gamma, end = 1 + gamma)
    p1b.x_range = Range1d(start = x_min, end = x_max)
    p1b.y_range = Range1d(start = 0, end = y_max)
    
    p2a.y_range = Range1d(start = 1 - gamma, end = 1 + gamma)
    p2b.x_range = Range1d(start = x_min, end = x_max)
    p2b.y_range = Range1d(start = 0, end = y_max)
    
    

    # # --- Названия панелей ---
    # p1a.title.text = "High dependence (l_ratio = 0.9)"
    # p2a.title.text = "Low dependence (l_ratio = 0.01)"
    # p1b.title.text = "Final estimates distribution (l_ratio = 0.9)"
    # p2b.title.text = "Final estimates distribution (l_ratio = 0.01)"

    # --- Компоновка: две строки, две колонки ---
    grid = gridplot([[p1a, p2a],
                     [p1b, p2b]],
                    toolbar_location=None,
                    merge_tools=False)

    show(grid)
    
    return p1a, p1b, p2a, p2b

In [51]:
p1a, p1b, p2a, p2b = experiment3_compare_l_ratios_sigma(alpha=1, a_true=1.0, n_max=100_000, steps=50,
                           n_trials=5, n_ci=1500, ci_level=0.95,
                                gamma = 0.1, x_min = 0.93, x_max = 1.07, y_max = 40)
save_pic(p = p1a, file_name = "sigma_alpha1_z_9")
save_pic(p = p1b, file_name = "sigma_hist_alpha1_z_9")
save_pic(p = p2a, file_name = "sigma_alpha1_z_01")
save_pic(p = p2b, file_name = "sigma_hist_alpha1_z_01")

100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 27.61it/s]


# Теоремы 3 и 5

In [53]:
# === Основная функция эксперимента === l(n) < n
def run_experiment4(a_true=1.0, n_max=10_000_000, steps=50, n_trials=5, l_ratio=0.5):
    """
    l_ratio ∈ (0, 1] — доля зависимых пар (X,Y)
    """
    n_vals = np.linspace(1000, n_max, steps, dtype=int)
    colors = pallets

    # --- Создаем два графика ---
    p1 = figure(
        width=500, height=400,
        # title=fr"Convergence of $\hat{{|a|}}_n$ to |a|={1}, up a_true = 1, down a_true = 1,",
        x_axis_label="Размер выборки n", y_axis_label=r"Оценка a^4",
        toolbar_location = None,
        tools="pan,wheel_zoom,box_zoom,reset,save",
        active_drag="box_zoom"
    )
    p1.add_tools(HoverTool(tooltips=[("n", "@x"), ("a_hat", "@y")]))
    p1.grid.grid_line_alpha = 0.4
    p1.xaxis.axis_label_text_font_size = "14pt"
    p1.yaxis.axis_label_text_font_size = "14pt"
    
    l_n = int(l_ratio * n_max)  # число зависимых пар
    l_n = max(1, min(l_n, n_max))  # защита от выхода за пределы

    # --- Эксперимент ---
    for trial in tqdm(range(n_trials)):
        color = colors[trial % len(colors)]

        # Чтобы не генерировать заново на каждом шаге n, можно использовать фиксированные массивы
        X_dep = laplace.rvs(size=l_n)
        eps_dep = norm.rvs(size=l_n)
        Y_dep = a_true * X_dep + eps_dep
        
        # независимые X и Y
        X_ind = laplace.rvs(size=n_max - l_n)
        Y_ind = a_true * laplace.rvs(size=n_max - l_n) + norm.rvs(size=n_max - l_n)
        
        # объединяем
        X_full = np.concatenate([X_dep, X_ind])
        Y_full = np.concatenate([Y_dep, Y_ind])
        
        X_full = np.random.permutation(X_full)
        Y_full = np.random.permutation(Y_full)

        a4 = []

        for n in n_vals:
            # --- Генерация данных ---
            # зависимые пары
            Xn = X_full[:n]
            Yn = Y_full[:n]

            # --- Вычисление оценок ---
            num4 = np.mean(Yn ** 4)
            den4 = np.mean(Xn ** 4)
            
            num2 = np.mean(Yn ** 2)
            den2 = np.mean(Xn ** 2)

            a4_ = safe_power_ratio2(num4, num2, den4, den2)
            a4.append(a4_)
            

        # --- Рисуем траектории ---
        p1.line(n_vals, a4, line_width=2, alpha=0.8, color=color)


    # --- Истинное значение ---
    p1.line([n_vals[0], n_vals[-1]], [a_true, a_true],
            color=pallets[-2], line_dash="dashed", line_width=2)
    
    

    # --- Эксперимент ---
    for trial in tqdm(range(n_trials)):
        color = colors[trial % len(colors)]

        # Чтобы не генерировать заново на каждом шаге n, можно использовать фиксированные массивы
        X_dep = laplace.rvs(size=l_n)
        eps_dep = norm.rvs(size=l_n)
        Y_dep = -a_true * X_dep + eps_dep
        
        # независимые X и Y
        X_ind = laplace.rvs(size=n_max - l_n)
        Y_ind = -a_true * laplace.rvs(size=n_max - l_n) + norm.rvs(size=n_max - l_n)
        
        # объединяем
        X_full = np.concatenate([X_dep, X_ind])
        Y_full = np.concatenate([Y_dep, Y_ind])
        
        X_full = np.random.permutation(X_full)
        Y_full = np.random.permutation(Y_full)

        a4 = []

        for n in n_vals:
            # --- Генерация данных ---
            # зависимые пары
            Xn = X_full[:n]
            Yn = Y_full[:n]

            # --- Вычисление оценок ---
            num4 = np.mean(Yn ** 4)
            den4 = np.mean(Xn ** 4)
            
            num2 = np.mean(Yn ** 2)
            den2 = np.mean(Xn ** 2)

            a4_ = safe_power_ratio2(num4, num2, den4, den2)
            a4.append(a4_)
            

        # --- Рисуем траектории ---
        p1.line(n_vals, -np.array(a4), line_width=2, alpha=0.8, color=color)


    # --- Истинное значение ---
    p1.line([n_vals[0], n_vals[-1]], [-a_true, -a_true],
            color=pallets[-2], line_dash="dashed", line_width=2)
    
    p2 = figure(
        width=500, height=400,
        # title="(empty)",
        x_axis_label="Оценка a^4", y_axis_label="Гистограмма",
        tools="pan,wheel_zoom,box_zoom,reset,save",
        toolbar_location = None,
        active_drag="box_zoom"
    )
    p2.grid.grid_line_alpha = 0.4
    p2.xaxis.axis_label_text_font_size = "14pt"
    p2.yaxis.axis_label_text_font_size = "14pt"

    layout = row(p1, p2)
    # show(p1)
    return layout


# === Функция с построением доверительного интервала и гистограммы ===
def run_experiment4_with_ci(a_true=1.0, n_max=40_000, steps=50,
                            n_trials=5, n_ci=30, l_ratio=0.5, ci_level=0.9):

    n_vals = np.linspace(1000, n_max, steps, dtype=int)

    all_trajectories = np.zeros((n_ci, len(n_vals)))
    
    l_n = int(l_ratio * n_max)  # число зависимых пар
    l_n = max(1, min(l_n, n_max))  # защита от выхода за пределы

    # --- Генерация траекторий для доверительного интервала ---
    for ci_trial in tqdm(range(n_ci), desc="CI simulations"):
        # Чтобы не генерировать заново на каждом шаге n, можно использовать фиксированные массивы
        X_dep = laplace.rvs(size=l_n)
        eps_dep = norm.rvs(size=l_n)
        Y_dep = a_true * X_dep + eps_dep
        
        # независимые X и Y
        X_ind = laplace.rvs(size=n_max - l_n)
        Y_ind = a_true * laplace.rvs(size=n_max - l_n) + norm.rvs(size=n_max - l_n)
        
        # объединяем
        X_full = np.concatenate([X_dep, X_ind])
        Y_full = np.concatenate([Y_dep, Y_ind])
        
        X_full = np.random.permutation(X_full)
        Y_full = np.random.permutation(Y_full)

        a4 = []

        for n in n_vals:
            # --- Генерация данных ---
            # зависимые пары
            Xn = X_full[:n]
            Yn = Y_full[:n]

            # --- Вычисление оценок ---
            num4 = np.mean(Yn ** 4)
            den4 = np.mean(Xn ** 4)
            
            num2 = np.mean(Yn ** 2)
            den2 = np.mean(Xn ** 2)

            a4_ = safe_power_ratio2(num4, num2, den4, den2)
            a4.append(a4_)

        all_trajectories[ci_trial, :] = a4

    # --- Квантильные интервалы ---
    lower_q = (1 - ci_level) / 2
    upper_q = 1 - lower_q
    q_lower = np.quantile(all_trajectories, lower_q, axis=0)
    q_upper = np.quantile(all_trajectories, upper_q, axis=0)

    # --- Основная фигура ---
    layout = run_experiment4(a_true=a_true,
                             n_max=n_max, steps=steps, n_trials=n_trials, l_ratio=l_ratio)
    p1, p2 = layout.children

    # --- Добавляем доверительный интервал ---
    source_band = ColumnDataSource(data=dict(
        x=n_vals, lower=q_lower, upper=q_upper
    ))

    band = Band(base='x', lower='lower', upper='upper', source=source_band,
                level='underlay', fill_alpha=0.25, fill_color=pallets[-1],
                line_width=1, line_color=pallets[-1])
    p1.add_layout(band)
    
    
    # --- Гистограмма последних оценок ---
    last_vals = all_trajectories[:, -1]

    # --- Гистограмма ---
    hist, edges = np.histogram(last_vals, bins=30, density=True)

    src_hist = ColumnDataSource(dict(
        left=edges[:-1],
        right=edges[1:],
        top=hist
    ))

    p2.quad(bottom=0, top='top', left='left', right='right',
            source=src_hist, fill_alpha=0.5, fill_color=pallets[-1],
            line_color='white')

    # --- Оценка среднего и стандартного отклонения ---
    mean_val = np.mean(last_vals)
    std_val = np.std(last_vals)

    # --- Плавная аппроксимация нормальной плотностью ---
    x_smooth = np.linspace(mean_val - 4*std_val, mean_val + 4*std_val, 400)
    y_smooth = norm.pdf(x_smooth, loc=mean_val, scale=std_val)

    p2.line(x_smooth, y_smooth, color=pallets[-1], line_width=3,
            line_dash="solid")

    # --- Вертикальные линии ---
    p2.line([a_true, a_true], [0, max(hist)], color=pallets[-2],
            line_dash="dashed", line_width=2, legend_label="True a^4")
    p2.line([mean_val, mean_val], [0, max(hist)], color=pallets[-3],
            line_dash="dotdash", line_width=2, legend_label="Mean \hat{a^4}")
    p2.line([mean_val - 3*std_val, mean_val - 3*std_val], [0, max(hist)], color=pallets[-3],
            line_dash="dotted", line_width=2, legend_label="3\hat{std}")
    p2.line([mean_val + 3*std_val, mean_val + 3*std_val], [0, max(hist)], color=pallets[-3],
            line_dash="dotted", line_width=2, legend_label="3\hat{std}")

    # --- Красивое оформление ---
    # p2.title.text = "Distribution of final estimates $\\hat{a}$"
    # p2.xaxis.axis_label = "â (final estimates)"
    # p2.yaxis.axis_label = "Density"
    p2.legend.location = "top_right"
    # p2.legend.background_fill_alpha = 0.4
    p2.legend.label_text_font_size = "10pt"
    p2.toolbar_location = None
    
    p2.legend.background_fill_alpha = 0     # убрать заливку (прозрачный фон)
    p2.legend.border_line_alpha = 0 
    
    
    
    for ci_trial in tqdm(range(n_ci), desc="CI simulations"):
        # Чтобы не генерировать заново на каждом шаге n, можно использовать фиксированные массивы
        X_dep = laplace.rvs(size=l_n)
        eps_dep = norm.rvs(size=l_n)
        Y_dep = -a_true * X_dep + eps_dep
        
        # независимые X и Y
        X_ind = laplace.rvs(size=n_max - l_n)
        Y_ind = -a_true * laplace.rvs(size=n_max - l_n) + norm.rvs(size=n_max - l_n)
        
        # объединяем
        X_full = np.concatenate([X_dep, X_ind])
        Y_full = np.concatenate([Y_dep, Y_ind])
        
        X_full = np.random.permutation(X_full)
        Y_full = np.random.permutation(Y_full)

        a4 = []

        for n in n_vals:
            # --- Генерация данных ---
            # зависимые пары
            Xn = X_full[:n]
            Yn = Y_full[:n]

            # --- Вычисление оценок ---
            num4 = np.mean(Yn ** 4)
            den4 = np.mean(Xn ** 4)
            
            num2 = np.mean(Yn ** 2)
            den2 = np.mean(Xn ** 2)

            a4_ = safe_power_ratio2(num4, num2, den4, den2)
            a4.append(a4_)

        all_trajectories[ci_trial, :] = -np.array(a4)

    # --- Квантильные интервалы ---
    lower_q = (1 - ci_level) / 2
    upper_q = 1 - lower_q
    q_lower = np.quantile(all_trajectories, lower_q, axis=0)
    q_upper = np.quantile(all_trajectories, upper_q, axis=0)

    # --- Добавляем доверительный интервал ---
    source_band = ColumnDataSource(data=dict(
        x=n_vals, lower=q_lower, upper=q_upper
    ))

    band = Band(base='x', lower='lower', upper='upper', source=source_band,
                level='underlay', fill_alpha=0.25, fill_color=pallets[-1],
                line_width=1, line_color=pallets[-1])
    p1.add_layout(band)
    
    
    

    # show(p1)
    return layout


def experiment4_compare_l_ratios(a_true=1.0, n_max=40_000, steps=50,
                                       n_trials=5, n_ci=30, ci_level=0.9,
                                gamma = 0.1, x_min = 0.9, x_max = 1.1, y_max = 70):
    """
    Строит два графика run_experiment3_with_ci с разными l_ratio
    и объединяет их в одну строку с синхронизированной осью Y,
    общими подписями и заголовком (стиль Figure 3 из статьи).
    """

    # --- Первая визуализация (почти зависимые данные) ---
    layout1 = run_experiment4_with_ci(
        a_true=a_true,
        n_max=n_max, steps=steps,
        n_trials=n_trials, n_ci=n_ci,
        l_ratio=0.9, ci_level=ci_level
    )

    # --- Вторая визуализация (почти независимые данные) ---
    layout2 = run_experiment4_with_ci(
        a_true=a_true,
        n_max=n_max, steps=steps,
        n_trials=n_trials, n_ci=n_ci,
        l_ratio=0.01, ci_level=ci_level
    )

    # --- Извлекаем подграфики ---
    p1a, p1b = layout1.children
    p2a, p2b = layout2.children

    # --- Извлекаем подграфики ---
    p1a, p1b = layout1.children
    p2a, p2b = layout2.children

    # # --- Синхронизация осей ---
    # p2a.x_range = p1a.x_range
    # p2a.y_range = p1a.y_range
    # p2b.x_range = p1b.x_range
    # p2b.y_range = p1b.y_range
    
    p1a.y_range = Range1d(start = -1 - gamma, end = 1 + gamma)
    p1b.x_range = Range1d(start = x_min, end = x_max)
    p1b.y_range = Range1d(start = 0, end = y_max)
    
    p2a.y_range = Range1d(start = -1 - gamma, end = 1 + gamma)
    p2b.x_range = Range1d(start = x_min, end = x_max)
    p2b.y_range = Range1d(start = 0, end = y_max)
    
    

    # # --- Названия панелей ---
    # p1a.title.text = "High dependence (l_ratio = 0.9)"
    # p2a.title.text = "Low dependence (l_ratio = 0.01)"
    # p1b.title.text = "Final estimates distribution (l_ratio = 0.9)"
    # p2b.title.text = "Final estimates distribution (l_ratio = 0.01)"

    # --- Компоновка: две строки, две колонки ---
    grid = gridplot([[p1a, p2a],
                     [p1b, p2b]],
                    toolbar_location=None,
                    merge_tools=False)

    show(grid)
    return p1a, p1b, p2a, p2b

    
    
    

In [54]:
    
# === Основная функция эксперимента === l(n) < n
def run_experiment5(a_true=1.0, n_max=100_000, steps=50, n_trials=5, l_ratio=0.5):
    """
    l_ratio ∈ (0, 1] — доля зависимых пар (X,Y)
    """
    n_vals = np.linspace(1000, n_max, steps, dtype=int)
    colors = pallets

    # --- Создаем два графика ---
    p1 = figure(
        width=500, height=400,
        # title=fr"Convergence of $\hat{{sigma^2}}_n$ to sigma^2, a_true = 1, l_ratio={l_ratio}",
        x_axis_label="Размер выборки n", y_axis_label=r"Оценка sigma^2",
        tools="pan,wheel_zoom,box_zoom,reset,save",
        toolbar_location = None,
        active_drag="box_zoom"
    )
    p1.add_tools(HoverTool(tooltips=[("n", "@x"), ("a_hat", "@y")]))
    p1.grid.grid_line_alpha = 0.4
    p1.xaxis.axis_label_text_font_size = "14pt"
    p1.yaxis.axis_label_text_font_size = "14pt"
    
    l_n = int(l_ratio * n_max)  # число зависимых пар
    l_n = max(1, min(l_n, n_max))  # защита от выхода за пределы

    # --- Эксперимент ---
    for trial in tqdm(range(n_trials)):
        color = colors[trial % len(colors)]

        X_dep = laplace.rvs(size=l_n)
        eps_dep = norm.rvs(size=l_n)
        Y_dep = a_true * X_dep + eps_dep
        
        # независимые X и Y
        X_ind = laplace.rvs(size=n_max - l_n)
        Y_ind = a_true * laplace.rvs(size=n_max - l_n) + norm.rvs(size=n_max - l_n)
        
        # объединяем
        X_full = np.concatenate([X_dep, X_ind])
        Y_full = np.concatenate([Y_dep, Y_ind])
        
        X_full = np.random.permutation(X_full)
        Y_full = np.random.permutation(Y_full)

        sigma2 = []

        for n in n_vals:
            
            # зависимые пары
            Xn = X_full[:n]
            Yn = Y_full[:n]

            # --- Вычисление оценок ---
            num4 = np.mean(Yn ** 4)
            den4 = np.mean(Xn ** 4)
            
            num2 = np.mean(Yn ** 2)
            den2 = np.mean(Xn ** 2)

            a4_ = safe_power_ratio2(num4, num2, den4, den2)
            if np.sign(a4_) < 0:
                idx = np.where(n_vals == n)[0][0]
                n_vals = np.delete(n_vals, idx)
                continue
            sigma2_ = num2 - den2 * a4_**(1/2)
            sigma2.append(sigma2_)
            

        # --- Рисуем траектории ---
        p1.line(n_vals, sigma2, line_width=2, alpha=0.8, color=color)


    # --- Истинное значение ---
    p1.line([n_vals[0], n_vals[-1]], [1, 1],
            color=pallets[-2], line_dash="dashed", line_width=2)
    
    p2 = figure(
        width=500, height=400,
        # title="(empty)",
        x_axis_label="Оценка sigma^2", y_axis_label="Гистограмма",
        toolbar_location = None,
        tools="pan,wheel_zoom,box_zoom,reset,save",
        active_drag="box_zoom"
    )
    p2.grid.grid_line_alpha = 0.4
    p2.xaxis.axis_label_text_font_size = "14pt"
    p2.yaxis.axis_label_text_font_size = "14pt"

    layout = row(p1, p2)
    # show(p1)
    return layout



# === Функция с построением доверительного интервала и гистограммы ===
def run_experiment5_with_ci(a_true=1.0, n_max=40_000, steps=50,
                            n_trials=5, n_ci=30, l_ratio=0.5, ci_level=0.9):

    n_vals = np.linspace(1000, n_max, steps, dtype=int)

    all_trajectories = np.zeros((n_ci, len(n_vals)))
    
    l_n = int(l_ratio * n_max)  # число зависимых пар
    l_n = max(1, min(l_n, n_max))  # защита от выхода за пределы

    # --- Генерация траекторий для доверительного интервала ---
    for ci_trial in tqdm(range(n_ci), desc="CI simulations"):
        # Чтобы не генерировать заново на каждом шаге n, можно использовать фиксированные массивы
        X_dep = laplace.rvs(size=l_n)
        eps_dep = norm.rvs(size=l_n)
        Y_dep = a_true * X_dep + eps_dep
        
        # независимые X и Y
        X_ind = laplace.rvs(size=n_max - l_n)
        Y_ind = a_true * laplace.rvs(size=n_max - l_n) + norm.rvs(size=n_max - l_n)
        
        # объединяем
        X_full = np.concatenate([X_dep, X_ind])
        Y_full = np.concatenate([Y_dep, Y_ind])
        
        X_full = np.random.permutation(X_full)
        Y_full = np.random.permutation(Y_full)

        sigma2 = []

        for n in n_vals:
            # зависимые пары
            Xn = X_full[:n]
            Yn = Y_full[:n]

            # --- Вычисление оценок ---
            num4 = np.mean(Yn ** 4)
            den4 = np.mean(Xn ** 4)
            
            num2 = np.mean(Yn ** 2)
            den2 = np.mean(Xn ** 2)

            a4_ = safe_power_ratio2(num4, num2, den4, den2)
            if np.sign(a4_) < 0:
                idx = np.where(n_vals == n)[0][0]
                n_vals = np.delete(n_vals, idx)
                continue
            sigma2_ = num2 - den2 * a4_**(1/2)
            sigma2.append(sigma2_)

        all_trajectories[ci_trial, :] = sigma2

    # --- Квантильные интервалы ---
    lower_q = (1 - ci_level) / 2
    upper_q = 1 - lower_q
    q_lower = np.quantile(all_trajectories, lower_q, axis=0)
    q_upper = np.quantile(all_trajectories, upper_q, axis=0)

    # --- Основная фигура ---
    layout = run_experiment5(a_true=a_true,
                             n_max=n_max, steps=steps, n_trials=n_trials, l_ratio=l_ratio)
    p1, p2 = layout.children

    # --- Добавляем доверительный интервал ---
    source_band = ColumnDataSource(data=dict(
        x=n_vals, lower=q_lower, upper=q_upper
    ))

    band = Band(base='x', lower='lower', upper='upper', source=source_band,
                level='underlay', fill_alpha=0.25, fill_color=pallets[-1],
                line_width=1, line_color=pallets[-1])
    p1.add_layout(band)
    
    
    # --- Гистограмма последних оценок ---
    last_vals = all_trajectories[:, -1]

    # --- Гистограмма ---
    hist, edges = np.histogram(last_vals, bins=30, density=True)

    src_hist = ColumnDataSource(dict(
        left=edges[:-1],
        right=edges[1:],
        top=hist
    ))

    p2.quad(bottom=0, top='top', left='left', right='right',
            source=src_hist, fill_alpha=0.5, fill_color=pallets[-1],
            line_color='white')

    # --- Оценка среднего и стандартного отклонения ---
    mean_val = np.mean(last_vals)
    std_val = np.std(last_vals)

    # --- Плавная аппроксимация нормальной плотностью ---
    x_smooth = np.linspace(mean_val - 4*std_val, mean_val + 4*std_val, 400)
    y_smooth = norm.pdf(x_smooth, loc=mean_val, scale=std_val)

    p2.line(x_smooth, y_smooth, color=pallets[-1], line_width=3,
            line_dash="solid")

    # --- Вертикальные линии ---
    p2.line([1, 1], [0, max(hist)], color=pallets[-2],
            line_dash="dashed", line_width=2, legend_label="True sigma^2")
    p2.line([mean_val, mean_val], [0, max(hist)], color=pallets[-3],
            line_dash="dotdash", line_width=2, legend_label="Mean \hat{sigma^2}")
    p2.line([mean_val - 3*std_val, mean_val - 3*std_val], [0, max(hist)], color=pallets[-3],
            line_dash="dotted", line_width=2, legend_label="3\hat{std}")
    p2.line([mean_val + 3*std_val, mean_val + 3*std_val], [0, max(hist)], color=pallets[-3],
            line_dash="dotted", line_width=2, legend_label="3\hat{std}")

    # --- Красивое оформление ---
    # p2.title.text = "Distribution of final estimates $\\hat{sigma^2}$"
    # p2.xaxis.axis_label = "sigma^2 (final estimates)"
    # p2.yaxis.axis_label = "Density"
    p2.legend.location = "top_right"
    # p2.legend.background_fill_alpha = 0.4
    p2.legend.label_text_font_size = "10pt"
    p2.toolbar_location = None
    
    p2.legend.background_fill_alpha = 0     # убрать заливку (прозрачный фон)
    p2.legend.border_line_alpha = 0 


    # show(p1)
    return layout


def experiment5_compare_l_ratios(a_true=1.0, n_max=40_000, steps=50,
                                n_trials=5, n_ci=30, ci_level=0.9,
                                gamma = 0.1, x_min = 0.9, x_max = 1.1, y_max = 70):
    """
    Строит два графика run_experiment3_with_ci с разными l_ratio
    и объединяет их в одну строку с синхронизированной осью Y,
    общими подписями и заголовком (стиль Figure 3 из статьи).
    """

    # --- Первая визуализация (почти зависимые данные) ---
    layout1 = run_experiment5_with_ci(
        a_true=a_true,
        n_max=n_max, steps=steps,
        n_trials=n_trials, n_ci=n_ci,
        l_ratio=0.9, ci_level=ci_level
    )

    # --- Вторая визуализация (почти независимые данные) ---
    layout2 = run_experiment5_with_ci(
        a_true=a_true,
        n_max=n_max, steps=steps,
        n_trials=n_trials, n_ci=n_ci,
        l_ratio=0.01, ci_level=ci_level
    )

    # --- Извлекаем подграфики ---
    p1a, p1b = layout1.children
    p2a, p2b = layout2.children

    # # --- Синхронизация осей ---
    # p2a.x_range = p1a.x_range
    # p2a.y_range = p1a.y_range
    # p2b.x_range = p1b.x_range
    # p2b.y_range = p1b.y_range
    
    p1a.y_range = Range1d(start = 1 - gamma, end = 1 + gamma)
    p1b.x_range = Range1d(start = x_min, end = x_max)
    p1b.y_range = Range1d(start = 0, end = y_max)
    
    p2a.y_range = Range1d(start = 1 - gamma, end = 1 + gamma)
    p2b.x_range = Range1d(start = x_min, end = x_max)
    p2b.y_range = Range1d(start = 0, end = y_max)
    
    

    # # --- Названия панелей ---
    # p1a.title.text = "High dependence (l_ratio = 0.9)"
    # p2a.title.text = "Low dependence (l_ratio = 0.01)"
    # p1b.title.text = "Final estimates distribution (l_ratio = 0.9)"
    # p2b.title.text = "Final estimates distribution (l_ratio = 0.01)"

    # --- Компоновка: две строки, две колонки ---
    grid = gridplot([[p1a, p2a],
                     [p1b, p2b]],
                    toolbar_location=None,
                    merge_tools=False)

    show(grid)
    return p1a, p1b, p2a, p2b

In [55]:
p1a, p1b, p2a, p2b = experiment4_compare_l_ratios(a_true=1.0, n_max=100_000, steps=50,
                                               n_trials=5, n_ci=1500, ci_level=0.95,
                                               gamma = 0.5, x_min = 0.74, x_max = 1.26, y_max = 15)
save_pic(p = p1a, file_name = "abs_a_alpha1_z_9")
save_pic(p = p1b, file_name = "abs_a_hist_alpha1_z_9")
save_pic(p = p2a, file_name = "abs_a_alpha1_z_01")
save_pic(p = p2b, file_name = "abs_a_hist_alpha1_z_01")

CI simulations: 100%|███████████████████████| 1500/1500 [01:40<00:00, 14.94it/s]


In [56]:
p1a, p1b, p2a, p2b = experiment5_compare_l_ratios(a_true=1.0, n_max=100_000, steps=50,
                                                  n_trials=5, n_ci=1500, ci_level=0.95,
                                                  gamma = 0.31, x_min = 0.81, x_max = 1.19, y_max = 15)
save_pic(p = p1a, file_name = "sigma_lap_z_9")
save_pic(p = p1b, file_name = "sigma_hist_lap_z_9")
save_pic(p = p2a, file_name = "sigma_lap_z_01")
save_pic(p = p2b, file_name = "sigma_hist_lap_z_01")

100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 14.63it/s]
